# Hyperparameter Tuning Analysis
Author(s): Cedric Donié (cedricdonie@gmail.com)

Analyzing the results of hyperparameter tuning experiments.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from cloudpathlib import AnyPath as Path
import json
from src.visualization.summarize_hyperparameter_tuning import parse_cv_run_filename
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import spearmanr, pearsonr

In [ ]:
plt.style.use(["science", "ieee", "../src/visualization/main.mplstyle", "vibrant"])

In [ ]:
def corrfunc(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, p = spearmanr(x, y)
    ax = ax or plt.gca()
    ax.annotate(Rf'$\rho = {r:.2f}$', xy=(.1, .9), xycoords=ax.transAxes)

In [ ]:
LABELS = {"depth": "Depth", "kernel_size": "Filter Length", "nb_filters": "Number of Filters", "window_length": "Window Length / s"}

In [ ]:
RESULTS_PATH = "../models/inceptiontime/hyperparameter_tuning/tremor/GENEActiv_train-val/results.csv"
MEAN_RESULTS_PATH = "../models/inceptiontime/hyperparameter_tuning/tremor/GENEActiv_train-val/meanresults.csv"
PHENOTYPE = "tremor"

In [ ]:
results = pd.read_csv(RESULTS_PATH)
mean_results = pd.read_csv(MEAN_RESULTS_PATH)

In [ ]:
FIGSAVEPATH = Path("../reports/figures/inceptiontime/hyperparameter_tuning/") / PHENOTYPE /"GENEActiv_train-val/"
FIGSAVEPATH.mkdir(exist_ok=True, parents=True)

In [ ]:
from matplotlib import patheffects

fig, axs = plt.subplots(nrows=2, ncols=2, sharex=False, sharey="row")
if PHENOTYPE == "tremor":
    for x, ax in zip(("depth", "kernel_size", "nb_filters", "window_length"), axs.flatten()):
        sns.scatterplot(data=results.dropna(), x=x, y="test_mAP", ax=ax, s=20)
        sns.regplot(data=mean_results.dropna(), x=x, y="test_mAP", ax=ax, color="C1", scatter_kws={"s": 20})
        rho, p = spearmanr(results.dropna()[x], results.dropna()["test_mAP"])
        text = ax.text(0.9, 0.1, rf"$r_s = {rho:.2f}$", transform=ax.transAxes, verticalalignment="bottom", horizontalalignment="right", bbox=dict(facecolor='white', alpha=0.5, edgecolor="none", pad=3))
        ax.set_xlabel(LABELS[x])
        ax.set_ylabel("mAP")
else:
    for x, ax in zip(("depth", "kernel_size", "nb_filters", "window_length"), axs.flatten()):
        sns.scatterplot(data=results.dropna(), x=x, y="test_average_precision", ax=ax, s=20)
        sns.regplot(data=mean_results.dropna(), x=x, y="test_average_precision", ax=ax, color="C1", scatter_kws={"s": 20})
        rho, p = spearmanr(results.dropna()[x], results.dropna()["test_average_precision"])
        text = ax.text(0.9, 0.1, rf"$r_s = {rho:.2f}$", transform=ax.transAxes, verticalalignment="bottom", horizontalalignment="right", bbox=dict(facecolor='white', alpha=0.5, edgecolor="none", pad=3))
        ax.set_xlabel(LABELS[x])
        ax.set_ylabel("AP")
fig.legend(labels=["_nolegend_", "all cross-validation folds", "cross-validation mean"], loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.1))
fig.savefig(FIGSAVEPATH / ("mAPoverall.pgf"))

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=4, sharex="col", sharey=True)
size = fig.get_size_inches()
for fig_row, phenotype in enumerate(("tremor", "bradykinesia", "dyskinesia")):
    results = pd.read_csv(f"../models/inceptiontime/hyperparameter_tuning/{phenotype}/GENEActiv_train-val/results.csv")
    mean_results = pd.read_csv(f"../models/inceptiontime/hyperparameter_tuning/{phenotype}/GENEActiv_train-val/meanresults.csv")
    for fig_col, x in enumerate(("window_length", "kernel_size", "nb_filters", "depth")):
        ax = axs[fig_row, fig_col]
        if phenotype == "tremor":
            sns.scatterplot(data=results.dropna(), x=x, y="test_mAP", ax=ax, s=20)
            sns.regplot(data=mean_results.dropna(), x=x, y="test_mAP", ax=ax, color="C1", scatter_kws={"s": 20}, ci=0)
            rho, p = spearmanr(results.dropna()[x], results.dropna()["test_mAP"])
            text = ax.text(0.9, 0.1, rf"$r_s = {rho:.2f}$", transform=ax.transAxes, verticalalignment="bottom", horizontalalignment="right", bbox=dict(facecolor='white', alpha=0.5, edgecolor="none", pad=3))
            ax.set_xlabel(LABELS[x])
            ax.set_ylabel("Tremor mAP")
        else:
            sns.scatterplot(data=results.dropna(), x=x, y="test_average_precision", ax=ax, s=20)
            sns.regplot(data=mean_results.dropna(), x=x, y="test_average_precision", ax=ax, color="C1", scatter_kws={"s": 20}, ci=0)
            rho, p = spearmanr(results.dropna()[x], results.dropna()["test_average_precision"])
            pos = (0.9, 0.7) if phenotype == "dyskinesia" else (0.9, 0.1)
            text = ax.text(pos[0], pos[1], rf"$r_s = {rho:.2f}$", transform=ax.transAxes, verticalalignment="bottom", horizontalalignment="right", bbox=dict(facecolor='white', alpha=0.5, edgecolor="none", pad=3))
            ax.set_xlabel(LABELS[x])
            ax.set_ylabel(f"{phenotype.title()} AP")
fig.set_size_inches((size[0] * 2, size[1] * 1.3))
fig.legend(labels=["_nolegend_", "all cross-validation folds", "cross-validation mean"], loc='upper center', ncol=2, bbox_to_anchor=(0.5, 1.1))
fig.savefig("../reports/figures/hypertuningscatter.pgf")

In [ ]:
if PHENOTYPE == "tremor":
    for x in ("depth", "kernel_size", "nb_filters"):
        fig, ax = plt.subplots()
        ax.set_xlabel(x)
        contour = ax.tricontourf(results[x], results["window_length"], results["test_mAP"])
        fig.colorbar(contour)
        fig2, ax2 = plt.subplots()
        g = sns.scatterplot(data=results, x=x, y="window_length", hue="test_mAP", ax=ax2)
        g.figure.savefig(FIGSAVEPATH / f"{x}-window_length-mAP.pgf")

In [ ]:
if PHENOTYPE == "tremor":
    import itertools
    for x, y, z in itertools.combinations(("depth", "kernel_size", "nb_filters", "window_length"), 3):
        fig = plt.figure()
        fig.set_size_inches(10, 10)
        ax = fig.add_subplot(projection='3d')
        ax.set_xlabel(x)
        ax.set_ylabel(y)
        ax.set_zlabel(z)
        print(x, y, z)
        ax.scatter(xs=results[x], ys=results[y], zs=results[z], c=results["test_mAP"])
        #ax.plot_trisurf(results[x], results[y], results[z])

In [ ]:
if PHENOTYPE == "tremor":
    fig = plt.figure()
    fig.set_size_inches(10, 10)
    ax = fig.add_subplot(projection="3d")
    ax.scatter(xs=mean_results["depth"], ys=mean_results["window_length"], zs=mean_results["test_mAP"])

In [ ]:
from scipy.stats import spearmanr
plt.style.use(["science", "../src/visualization/main.mplstyle", "../src/visualization/smallfigures.mplstyle"])

results2 = results.copy()
results2.columns = results2.columns.str.replace("test_", "")
results2.columns = results2.columns.str.replace("_", " ")
results2.columns = results2.columns.str.title()
results2.columns = results2.columns.str.replace("Map", "mAP")
results2.columns = results2.columns.str.replace("Average Precision", "AP")
x = "mAP" if PHENOTYPE == "tremor" else "AP"
ax = sns.regplot(x=x, y="Balanced Accuracy", data=results2.dropna())
rho, p = spearmanr(results2.dropna()[x], results2.dropna()["Balanced Accuracy"])
text = ax.text(0.05, 0.8, rf"$r_s = {rho:.2f}$", transform=ax.transAxes, verticalalignment="bottom", horizontalalignment="left")
ax.figure.savefig(FIGSAVEPATH / "baovermAP.pgf", bbox_inches="tight")